# Traffic Analysis Project

# Traffic Analysis Project


## 1. Data Preprocessing


In [ ]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'Banglore_traffic_Dataset 2.csv'
df = pd.read_csv(file_path)

# Convert 'Date' column to datetime format and extract time-based features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday

# Encode categorical columns ('Weather Conditions' and 'Roadwork and Construction Activity')
label_encoder = LabelEncoder()
df['Weather Conditions'] = label_encoder.fit_transform(df['Weather Conditions'])
df['Roadwork and Construction Activity'] = label_encoder.fit_transform(df['Roadwork and Construction Activity'])

# Drop the original 'Date' column
df.drop(columns=['Date'], inplace=True)

# Check for missing values
df.isnull().sum()



## 2. Exploratory Data Analysis (EDA)


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Visualizing the distribution of Traffic Volume
plt.figure(figsize=(10, 6))
sns.histplot(df['Traffic Volume'], kde=True, bins=30)
plt.title('Distribution of Traffic Volume')
plt.xlabel('Traffic Volume')
plt.ylabel('Frequency')
plt.show()

# Correlation heatmap to identify relationships
correlation_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Features')
plt.show()

# Scatter plot for Traffic Volume vs Average Speed
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Average Speed', y='Traffic Volume', data=df)
plt.title('Traffic Volume vs Average Speed')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Traffic Volume')
plt.show()

# Scatter plot for Traffic Volume vs Congestion Level
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Congestion Level', y='Traffic Volume', data=df)
plt.title('Traffic Volume vs Congestion Level')
plt.xlabel('Congestion Level')
plt.ylabel('Traffic Volume')
plt.show()



## 3. Model Building
### Traffic Volume Prediction - Random Forest Regressor


In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into train and test sets
X = df.drop(columns=['Traffic Volume', 'Congestion Level', 'Incident Reports'])
y_traffic_volume = df['Traffic Volume']

X_train, X_test, y_train, y_test = train_test_split(X, y_traffic_volume, test_size=0.2, random_state=42)

# Initialize Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Predict on test data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

(mse, rmse, r2)



### Congestion Level Classification - Random Forest Classifier


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Categorize congestion levels into Low, Medium, High
congestion_labels = ['Low', 'Medium', 'High']
df['Congestion Level Category'] = pd.cut(df['Congestion Level'], bins=[0, 33, 66, 100], labels=congestion_labels)

# Separate features and target variable for classification
y_congestion = df['Congestion Level Category']
X_classification = df.drop(columns=['Congestion Level', 'Traffic Volume', 'Incident Reports', 'Congestion Level Category'])

# Split the data for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_classification, y_congestion, test_size=0.2, random_state=42)

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_classifier.fit(X_train_class, y_train_class)

# Predict on test data
y_pred_class = rf_classifier.predict(X_test_class)

# Evaluate the model
classification_rep = classification_report(y_test_class, y_pred_class, target_names=congestion_labels)

classification_rep



### Incident Detection - Random Forest Classifier


In [ ]:

# Prepare the data for incident detection
y_incident = df['Incident Reports']
X_incident_classification = df.drop(columns=['Congestion Level', 'Traffic Volume', 'Incident Reports'])

# Split the data for incident detection
X_train_incident, X_test_incident, y_train_incident, y_test_incident = train_test_split(X_incident_classification, y_incident, test_size=0.2, random_state=42)

# Standardize numerical features for incident detection
scaler = StandardScaler()
X_train_incident_scaled = scaler.fit_transform(X_train_incident)
X_test_incident_scaled = scaler.transform(X_test_incident)

# Initialize Random Forest Classifier for incident detection
rf_incident_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_incident_classifier.fit(X_train_incident_scaled, y_train_incident)

# Predict on the test data
y_pred_incident = rf_incident_classifier.predict(X_test_incident_scaled)

# Evaluate the model performance
incident_classification_report = classification_report(y_test_incident, y_pred_incident)

incident_classification_report



## 4. Traffic Flow Optimization - KMeans Clustering


In [ ]:

from sklearn.cluster import KMeans

# Select only the numeric columns for clustering
X_clustering = df.select_dtypes(include=['float64', 'int64'])

# Normalize the features for clustering
scaler_for_clustering = StandardScaler()
X_clustering_scaled = scaler_for_clustering.fit_transform(X_clustering)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_clustering_scaled)

# Visualize the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Average Speed', y='Traffic Volume', hue='Cluster', palette='Set1', data=df, s=100, alpha=0.7)
plt.title('Traffic Clusters Based on Traffic Volume and Average Speed')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Traffic Volume')
plt.legend(title='Cluster')
plt.show()

# Analyze cluster centers
cluster_centers = kmeans.cluster_centers_
cluster_centers_df = pd.DataFrame(cluster_centers, columns=X_clustering.columns)
cluster_centers_df



## 5. Model Evaluation and Fine-tuning


In [1]:

## 5. Model Evaluation and Fine-tuning
# In this section, we will evaluate the models using various metrics like **Accuracy**, **F1-Score**, **RMSE**, and **MAE**. Additionally, we will fine-tune the models using techniques like **GridSearchCV** to improve performance.


# Add code to fine-tune models using GridSearchCV

from sklearn.model_selection import GridSearchCV

# Fine-tuning Random Forest Regressor for Traffic Volume Prediction
param_grid_rf_regressor = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}
grid_search_rf_regressor = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf_regressor, cv=3, scoring='neg_mean_squared_error')
grid_search_rf_regressor.fit(X_train, y_train)

best_rf_regressor = grid_search_rf_regressor.best_estimator_
y_pred_best_rf_regressor = best_rf_regressor.predict(X_test)
rmse_best_rf_regressor = mean_squared_error(y_test, y_pred_best_rf_regressor, squared=False)

# Fine-tuning Random Forest Classifier for Congestion Level Classification
param_grid_rf_classifier = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}
grid_search_rf_classifier = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf_classifier, cv=3, scoring='accuracy')
grid_search_rf_classifier.fit(X_train_class, y_train_class)

best_rf_classifier = grid_search_rf_classifier.best_estimator_
y_pred_best_rf_classifier = best_rf_classifier.predict(X_test_class)
classification_rep_best_rf_classifier = classification_report(y_test_class, y_pred_best_rf_classifier, target_names=congestion_labels)

rmse_best_rf_regressor, classification_rep_best_rf_classifier


# Add the evaluation cells to the notebook
nb.cells.append(model_evaluation_cell)
nb.cells.append(model_evaluation_code)




NameError: name 'RandomForestRegressor' is not defined

In [2]:
import shutil

# Move file to a common directory for download
shutil.copy("/mnt/data/Satarupa_fixed.ipynb", "./Satarupa_fixed.ipynb")

print("File is ready to download. Check your working directory.")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Satarupa_fixed.ipynb'